# Model

## Libs

In [2]:
import gc
import json
import pandas as pd
import unicodedata

import scipy.sparse
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
import joblib
import os

## Data

In [3]:
Data_path = '../data/json/'

In [4]:
with open(Data_path + 'applicants.json', 'r', encoding='utf-8') as file:
	applicants = json.load(file)

rows = []
for applicant in applicants:
	id = applicant
	informacoes_pessoais = applicants.get(id).get("informacoes_pessoais")
	informacoes_profissionais = applicants.get(id).get("informacoes_profissionais")
	formacao_e_idiomas = applicants.get(id).get("formacao_e_idiomas")
	aux = {
		'id': int(id),
		'pcd': informacoes_pessoais.get('pcd'),
		'titulo_profissional': informacoes_profissionais.get('titulo_profissional'),
		'area_atuacao': informacoes_profissionais.get('area_atuacao'),
		'conhecimentos_tecnicos': informacoes_profissionais.get('conhecimentos_tecnicos'),
		'certificacoes': informacoes_profissionais.get('certificacoes'),
		'outras_certificacoes': informacoes_profissionais.get('outras_certificacoes'),
		'nivel_academico': formacao_e_idiomas.get('nivel_academico'),
		'nivel_ingles': formacao_e_idiomas.get('nivel_ingles'),
		'nivel_espanhol': formacao_e_idiomas.get('nivel_espanhol'),
		'outro_idioma': formacao_e_idiomas.get('outro_idioma'),
		'cv_pt': applicants.get(id).get("cv_pt"),
	}
	rows.append(aux)

applicants_df = pd.DataFrame(data=rows, columns=['id', 'pcd', 'endereco', 'titulo_profissional', 'area_atuacao', 'conhecimentos_tecnicos','certificacoes', 'outras_certificacoes', 'nivel_academico', 'nivel_ingles', 'nivel_espanhol', 'outro_idioma', 'cv_pt'])

In [5]:
with open(Data_path + 'vagas.json', 'r', encoding='utf-8') as file:
	vagas = json.load(file)

rows = []
for vaga in vagas:
	id = vaga
	perfil_vaga = vagas.get(id).get("perfil_vaga")
	aux = {
		'id': int(id),
		'vaga_especifica_para_pcd': perfil_vaga.get('vaga_especifica_para_pcd'),
		'nivel_profissional': perfil_vaga.get('nivel profissional'),
		'nivel_academico': perfil_vaga.get('nivel_academico'),
		'nivel_ingles': perfil_vaga.get('nivel_ingles'),
		'nivel_espanhol': perfil_vaga.get('nivel_espanhol'),
		'outro_idioma': perfil_vaga.get('outro_idioma'),
		'areas_atuacao': perfil_vaga.get('areas_atuacao'),
		'principais_atividades': perfil_vaga.get('principais_atividades'),
		'demais_observacoes': perfil_vaga.get('demais_observacoes'),
		'viagens_requeridas': perfil_vaga.get('viagens_requeridas')
	}
	rows.append(aux)

vagas_df = pd.DataFrame(data=rows, columns=['id', 'estado', 'vaga_especifica_para_pcd', 'nivel_profissional', 'nivel_academico', 'nivel_ingles', 'nivel_espanhol', 'outro_idioma', 'areas_atuacao', 'principais_atividades', 'demais_observacoes', 'viagens_requeridas'])

In [6]:
with open(Data_path + 'prospects.json', 'r', encoding='utf-8') as file:
	prospects = json.load(file)

rows = []
for prospect in prospects:
	id = prospect
	candidates = prospects.get(id).get("prospects")
	for candidate in candidates:
		aux_id:int = int(id)
		aux = {
			'id_vaga': aux_id,
			'id_applicant': int(candidate.get("codigo")),
			'situacao_candidado': candidate.get("situacao_candidado"),
			'comentario': candidate.get("comentario")
		}
		rows.append(aux)

prospects_df = pd.DataFrame(data=rows, columns=['id_vaga','id_applicant', 'situacao_candidado', 'comentario'])

In [7]:
aux_merge_ap = pd.merge(applicants_df, prospects_df, left_on='id', right_on='id_applicant', how='inner')
merge = pd.merge(aux_merge_ap, vagas_df, left_on='id_vaga', right_on='id', how='inner', suffixes=('', '_vaga'))

df = merge[[
    'pcd',
    'titulo_profissional',
    'area_atuacao',
    'conhecimentos_tecnicos',
    'certificacoes',
    'outras_certificacoes',
    'nivel_academico',
    'nivel_ingles',
    'nivel_espanhol',
    'outro_idioma',
    'cv_pt',
    'situacao_candidado',
    'comentario',
    'vaga_especifica_para_pcd',
    'nivel_profissional',
    'nivel_academico_vaga',
    'nivel_ingles_vaga',
    'nivel_espanhol_vaga',
    'outro_idioma_vaga',
    'areas_atuacao',
    'principais_atividades',
    'demais_observacoes',
    'viagens_requeridas'
]]

df = df.drop_duplicates()

In [8]:
del applicants
del vagas
del prospects
del applicants_df
del prospects_df
del vagas_df
del aux_merge_ap
del merge
gc.collect()

0

## Data Pre Processor

In [9]:
nivel_map = {
    'nenhum': 0,
    'basico': 1,
    'intermediario': 2,
    'avancado': 3,
    'fluente': 4
}

situacao_map = {
    "candidatura em andamento": [
        "inscrito",
        "prospect",
        "em avaliacao pelo rh",
        "entrevista tecnica",
        "entrevista com cliente",
        "encaminhado ao requisitante",
        "encaminhar proposta",
        "documentacao pj",
        "documentacao clt",
        "documentacao cooperado"
    ],
    "contratacao confirmada": [
        "aprovado",
        "proposta aceita",
        "contratado pela decision",
        "contratado como hunting"
    ],
    "recusado": [
        "nao aprovado pelo rh",
        "nao aprovado pelo requisitante",
        "nao aprovado pelo cliente"
    ],
    "desistencia": [
        "desistiu",
        "desistiu da contratacao",
        "recusado",
        "sem interesse nesta vaga"
    ]
}

In [10]:
def string_normalizer(text):
	if text is None or text == "":
		return None
	else:
		return ''.join(c for c in unicodedata.normalize('NFD', text) if not unicodedata.combining(c)).lower()

def status_map(text):
    for grupo, status_list in situacao_map.items():
        if text in status_list:
            return grupo
    return None

In [11]:
df = df.apply(lambda col: col.map(string_normalizer) if col.dtype == 'object' else col)
df.replace('-', None, inplace=True)

In [12]:
df['pcd'] = df['pcd'].apply(lambda x: 1 if x == 'sim' else 0)
df['vaga_especifica_para_pcd'] = df['vaga_especifica_para_pcd'].apply(lambda x: 1 if x == 'sim' else 0)
df['pcd_match'] = ((df['pcd'] == 1) & (df['vaga_especifica_para_pcd'] == 1)).astype(int)

In [13]:
df['viagens_requeridas'] = df['viagens_requeridas'].apply(lambda x: 1 if x == 'sim' else 0)

In [14]:
df['nivel_ingles'] = df['nivel_ingles'].map(nivel_map).fillna(0)
df['nivel_ingles_vaga'] = df['nivel_ingles_vaga'].map(nivel_map).fillna(0)
df['ingles'] = (df['nivel_ingles'] >= df['nivel_ingles_vaga']).astype(int)

In [15]:
df['nivel_espanhol'] = df['nivel_ingles'].map(nivel_map).fillna(0)
df['nivel_espanhol_vaga'] = df['nivel_ingles_vaga'].map(nivel_map).fillna(0)
df['espanhol'] = (df['nivel_espanhol'] >= df['nivel_espanhol_vaga']).astype(int)

In [16]:
df['situacao_candidado'] = df['situacao_candidado'].map(status_map)

In [17]:
df.fillna("", inplace=True)

In [18]:
candidato_cols = ['titulo_profissional', 'area_atuacao', 'conhecimentos_tecnicos', 'certificacoes', 'outras_certificacoes', 'nivel_academico', 'outro_idioma', 'cv_pt']

df['candidato'] = df[candidato_cols].agg(' '.join, axis=1)

In [19]:
vaga_cols = ['nivel_profissional', 'nivel_academico_vaga', 'outro_idioma_vaga', 'areas_atuacao', 'principais_atividades', 'demais_observacoes']

df['vaga'] = df[vaga_cols].agg(' '.join, axis=1)

In [20]:
df = df[['candidato', 'vaga', 'viagens_requeridas', 'pcd_match', 'ingles', 'espanhol', 'comentario', 'situacao_candidado']]

In [21]:
df.rename(columns={
    'situacao_candidado': 'situacao',
    'comentario': 'comentario',
    'viagens_requeridas': 'vaga_viagens_requeridas',
    'pcd_match': 'pcd',
    'ingles': 'nivel_ingles',
    'espanhol': 'nivel_espanhol'
}, inplace=True)

## Models

In [22]:
df_model = df.copy()

In [23]:
print(df_model.info())
print(df_model.columns)

<class 'pandas.core.frame.DataFrame'>
Index: 44780 entries, 0 to 45070
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   candidato                44780 non-null  object
 1   vaga                     44780 non-null  object
 2   vaga_viagens_requeridas  44780 non-null  int64 
 3   pcd                      44780 non-null  int64 
 4   nivel_ingles             44780 non-null  int64 
 5   nivel_espanhol           44780 non-null  int64 
 6   comentario               44780 non-null  object
 7   situacao                 44780 non-null  object
dtypes: int64(4), object(4)
memory usage: 3.1+ MB
None
Index(['candidato', 'vaga', 'vaga_viagens_requeridas', 'pcd', 'nivel_ingles',
       'nivel_espanhol', 'comentario', 'situacao'],
      dtype='object')


In [24]:
label_encoders = {}
le = LabelEncoder()
df_model['situacao'] = le.fit_transform(df_model['situacao'])
label_encoders['situacao'] = le

In [25]:
vectorizer_candidato = TfidfVectorizer(max_features=3000)
vt_candidato = vectorizer_candidato.fit_transform(df_model['candidato'])

In [26]:
vectorizer_vaga = TfidfVectorizer(max_features=3000)
vt_vaga = vectorizer_candidato.fit_transform(df_model['vaga'])

In [27]:
vectorizer_comentario = TfidfVectorizer(max_features=1000)
vt_comentario = vectorizer_candidato.fit_transform(df_model['comentario'])

In [28]:
aux = df_model[['vaga_viagens_requeridas', 'pcd', 'nivel_ingles', 'nivel_espanhol']]
scaler = StandardScaler()
num_scaled = scaler.fit_transform(aux)

In [29]:

X = scipy.sparse.hstack([num_scaled, vt_candidato, vt_vaga, vt_comentario])
y = df_model['situacao']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

### Random Florest

#### Model

In [30]:
rf = RandomForestClassifier(n_estimators=200, random_state=42, max_depth=20, class_weight='balanced')

#### Train

In [31]:
rf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=20, n_estimators=200,
                       random_state=42)

#### Predict

In [32]:
y_pred_rf = rf.predict(X_test)

print(classification_report(y_test, y_pred_rf, zero_division=0))

              precision    recall  f1-score   support

           0       0.93      0.94      0.93      8657
           1       0.39      0.57      0.46       626
           2       0.78      0.56      0.65       617
           3       0.81      0.73      0.77      1295

    accuracy                           0.87     11195
   macro avg       0.73      0.70      0.70     11195
weighted avg       0.88      0.87      0.87     11195



### XGBoost

#### Model

In [33]:
xgb = XGBClassifier(
    objective='multi:softmax',
    num_class=len(y.unique()),
    eval_metric='mlogloss',
    random_state=42,
    n_estimators=200,
    max_depth=20,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
	min_child_weight=2
)

#### Train

In [34]:
xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=20, max_leaves=None,
              min_child_weight=2, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None, num_class=4, ...)

#### Predict

In [35]:
y_pred_xgb = xgb.predict(X_test)
print(classification_report(y_test, y_pred_xgb, zero_division=0))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96      8657
           1       0.78      0.39      0.52       626
           2       0.84      0.59      0.69       617
           3       0.83      0.79      0.81      1295

    accuracy                           0.91     11195
   macro avg       0.84      0.69      0.74     11195
weighted avg       0.90      0.91      0.90     11195



## Metrics

## Save Model

In [36]:
def mkpath(path: str):
    if not os.path.exists(path):
        os.makedirs(path)

### Random Forest

In [37]:
rf_path = "./RandomForest/"

mkpath(rf_path)

joblib.dump(rf, rf_path + "model.gz")
joblib.dump(label_encoders, rf_path + "label_encoders.gz")
joblib.dump(vectorizer_candidato, rf_path + "vectorizer_candidato.gz")
joblib.dump(vectorizer_vaga, rf_path + "vectorizer_vaga.gz")
joblib.dump(vectorizer_comentario, rf_path + "vectorizer_comentario.gz")
joblib.dump(scaler, rf_path + "standard_scaler.gz")

['./RandomForest/standard_scaler.gz']

### XGBoost

In [38]:
xgb_path = "./XGBoost/"

mkpath(xgb_path)

joblib.dump(xgb, xgb_path + "model.gz")
joblib.dump(label_encoders, xgb_path + "label_encoders.gz")
joblib.dump(vectorizer_candidato, xgb_path + "vectorizer_candidato.gz")
joblib.dump(vectorizer_vaga, xgb_path + "vectorizer_vaga.gz")
joblib.dump(vectorizer_comentario, xgb_path + "vectorizer_comentario.gz")
joblib.dump(scaler, xgb_path + "standard_scaler.gz")

['./XGBoost/standard_scaler.gz']